In [2]:
# -*- coding: utf-8 -*-
"""
Created on Tue Nov 22 17:09:50 2022

@author: SNUAI
"""

# !pip install -q git+https://github.com/tensorflow/docs
# import tensorflow_docs as tfdocs
# import tensorflow_docs.modeling
# import tensorflow_docs.plots

import pathlib
import shutil
import tempfile

import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Dense, Flatten, Conv2D,MaxPooling2D
from tensorflow.keras import Model
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
import os
plt.rcParams['figure.figsize'] = (12.0, 6.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)

path = './total_data'
img_height =128
img_width =128 
batch_size = 32
train_dataset = tf.keras.utils.image_dataset_from_directory(
    path,
    validation_split=0.2,
    subset="training",
    seed=123,
    color_mode = 'grayscale',
    image_size=(img_height, img_width),
    batch_size=batch_size)

valid_dataset = tf.keras.utils.image_dataset_from_directory(
  path,
  validation_split=0.2,
  subset="validation",
  seed=123,
  color_mode = 'grayscale',
  image_size=(img_height, img_width),
  batch_size=batch_size)

def regularized_padded_conv(*args, **kwargs):
    return tf.keras.layers.Conv2D(*args, **kwargs, padding='same', kernel_regularizer=_regularizer,
                                  kernel_initializer='he_normal', use_bias=False)
    
def regularized_padded_conv3(*args, **kwargs):
    return tf.keras.layers.Conv2D(*args, **kwargs, kernel_size=3, kernel_regularizer=_regularizer,
                                  padding='same', kernel_initializer='he_normal')
    
def bn_relu(x):
    x = tf.keras.layers.BatchNormalization()(x)
    return tf.keras.layers.ReLU()(x)

## Shortcut = skip connection

def shortcut(x, filters, stride):
    if x.shape[-1] == filters:
        return x
    else:
        x = regularized_padded_conv(filters, 1, strides=stride)(x)
        return x

def original_block(x, filters, stride=1, **kwargs):
    c1 = regularized_padded_conv(filters, 3, strides=stride)(x)
    c2 = regularized_padded_conv(filters, 3)(bn_relu(c1))
    c2 = tf.keras.layers.BatchNormalization()(c2)
    
    x = shortcut(x, filters, stride)
    return tf.keras.layers.ReLU()(x + c2)
    
def group_of_blocks(x,  num_blocks, filters, stride):
    x = original_block(x, filters, stride)
    for i in range(num_blocks-1):
        x = original_block(x, filters)
    return x

## Resnet
def Resnet(input_shape, 
           n_classes, 
           l2_reg=1e-4, 
           group_sizes=(64,128,256,512), 
           features=(1, 2, 2, 2), 
           strides=(1, 2, 2, 2),
            first_conv={"filters": 64, "kernel_size": 3, "strides": 1}):
    global _regularizer
    _regularizer = tf.keras.regularizers.l2(l2_reg)
    
    inputs = tf.keras.layers.Input(shape=input_shape)
    flow = regularized_padded_conv(**first_conv)(inputs)
    
    flow = bn_relu(flow)
    
    for block_idx, (group_size, feature, stride) in enumerate(zip(group_sizes, features, strides)):
        flow = group_of_blocks(flow,
                               num_blocks=group_size,
                               filters=feature,
                               stride=stride)
    
    flow = bn_relu(flow)
    
    flow = tf.keras.layers.GlobalAveragePooling2D()(flow)
    outputs = tf.keras.layers.Dense(n_classes, kernel_regularizer=_regularizer)(flow)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# model 생성
def resnet18(l2_reg=1e-4, load_weights=False):
    model = Resnet(input_shape=(128, 128, 1), n_classes=1, l2_reg=l2_reg, group_sizes=(2,2,2,2), features=(64,128,256,512),
                   strides=(1, 2, 2, 2), first_conv={"filters": 64, "kernel_size": 3, "strides": 1})
    return model


Found 2211 files belonging to 2 classes.
Using 1769 files for training.
Found 2211 files belonging to 2 classes.
Using 442 files for validation.


In [3]:

## 모델학습
tf.keras.backend.clear_session()
RESNET = resnet18()

RESNET.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

RESNET.fit(train_dataset, validation_data=valid_dataset, epochs=20)
RESNET.evaluate(valid_dataset)

Epoch 1/20
56/56 [==============================] - 25s 330ms/step - loss: 1.2271 - accuracy: 0.8615 - val_loss: 41.2488 - val_accuracy: 0.3507
Epoch 2/20
56/56 [==============================] - 16s 284ms/step - loss: 1.0217 - accuracy: 0.9633 - val_loss: 4.9512 - val_accuracy: 0.3507
Epoch 3/20
56/56 [==============================] - 16s 285ms/step - loss: 0.9637 - accuracy: 0.9853 - val_loss: 3.5860 - val_accuracy: 0.4434
Epoch 4/20
56/56 [==============================] - 16s 286ms/step - loss: 0.9300 - accuracy: 0.9938 - val_loss: 1.7231 - val_accuracy: 0.8167
Epoch 5/20
56/56 [==============================] - 16s 291ms/step - loss: 0.8981 - accuracy: 0.9977 - val_loss: 1.0531 - val_accuracy: 0.9434
Epoch 6/20
56/56 [==============================] - 17s 297ms/step - loss: 0.8657 - accuracy: 1.0000 - val_loss: 0.9379 - val_accuracy: 0.9661
Epoch 7/20
56/56 [==============================] - 16s 294ms/step - loss: 0.8472 - accuracy: 1.0000 - val_loss: 0.9261 - val_accuracy: 0.961

[0.6853058338165283, 0.9773755669593811]

In [4]:
print("done")

done
